### 1. combine `mon_features.pkl` & `unmon_features.pkl` into `features_df`


In [1]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '../content/mon_features.pkl'
UNMON_FILE_PATH = '../content/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER1 = [
    'total_transmission_time', 'std_inter_packet_time',
    'avg_outgoing_burst_size', 'avg_incoming_burst_size',
    'num_outgoing_packets', 'cumul_max',
    'outgoing_first_30', 'avg_incoming_order_first_30'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER1]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)


       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
28995                    32.09               0.163669   
28996                    38.62               0.114350   
28997                    34.93               1.331199   
28998                    11.84               0.083521   
28999                     9.62               0.026874   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

### 2. Random Forest


Split the dataset into training and testing sets


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Train and test Random Forest


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

clf_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)

print("============ Before Hyperparameter tuning ============")
print("RF Accuracy: {}".format(accuracy_score(y_test, y_pred_rf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


============ Before Hyperparameter tuning ============
RF Accuracy: 0.8337931034482758
Classification Report:
               precision    recall  f1-score   support

          -1       0.84      0.99      0.91      2470
           0       0.83      0.71      0.77        42
           1       0.95      0.75      0.84        51
           2       0.89      0.70      0.79        47
           3       0.94      0.57      0.71        51
           4       0.78      0.81      0.80        43
           5       0.88      0.84      0.86        44
           6       0.90      0.98      0.94        53
           7       0.70      0.67      0.69        55
           8       0.82      0.79      0.80        47
           9       0.81      0.64      0.72        45
          10       0.82      0.70      0.76        44
          11       0.93      0.67      0.78        55
          12       0.88      0.87      0.87        52
          13       0.72      0.40      0.52        57
          14       0.78 

### 3. Hyperparameter tuning by using Grid Search


In [4]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

search = HalvingGridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1), 
                             param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train, y_train)
y_pred_tuned = search.predict(X_test)

print("============ After Hyperparameter tuning ============")
print("Best Parameters:", search.best_params_)
print("RF Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))


============ After Hyperparameter tuning ============
Best Parameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
RF Accuracy: 0.852
Classification Report:
               precision    recall  f1-score   support

          -1       0.92      0.97      0.94      2470
           0       0.80      0.76      0.78        42
           1       0.91      0.84      0.88        51
           2       0.82      0.77      0.79        47
           3       0.92      0.69      0.79        51
           4       0.80      0.84      0.82        43
           5       0.88      0.84      0.86        44
           6       0.86      0.96      0.91        53
           7       0.78      0.71      0.74        55
           8       0.83      0.72      0.77        47
           9       0.78      0.69      0.73        45
          10       0.79      0.70      0.75        44
          11       0.86      0.78      0.82        55
          12       